**Loading the google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Setting the directory path**

In [ ]:
%cd /content/gdrive/My\ Drive/math_seq2tree/

/content/gdrive/My Drive/math_seq2tree


**Installing the requirements**

In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


**Changing the PyTorch Version**

In [ ]:
!pip uninstall torch
!pip install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl

Uninstalling torch-1.7.0+cu101:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.6/dist-packages/caffe2/*
    /usr/local/lib/python3.6/dist-packages/torch-1.7.0+cu101.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.7.0+cu101
     |████████████████████████████████| 636.8MB 28kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.0.1.post2 which is incompatible.


In [ ]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 5.8MB/s 
     |████████████████████████████████| 133kB 18.2MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 6.9MB 13.7MB/s 
ERROR: botocore 1.19.25 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


**Original Model with embedding size 128**

In [ ]:
!python3 run_seq2tree.py

Reading lines...
Transfer numbers...
(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['(', 'N1', '-', '1', ')', '*', 'N0'], ['2', '11'], [16, 34])
Indexing words...
keep_words 3671 / 9971 = 0.3682
Vocab Saved
Indexed 3674 words in input language, 23 words in output
Number of training data 18530
Number of testind data 4632
([3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 10, 15, 1, 16, 17, 2, 10, 18, 19, 20, 21, 16, 22], 27, [0, 9, 8], 3, ['5', '0.8', '150'], [3, 12, 16], [])
fold: 1
epoch: 1
loss: 1.3497868589286146
training time 0h 2m 36s
--------------------------------
1119 1265 4632
test_answer_acc 0.241580310880829 0.27310017271157166
testing time 0h 5m 28s
------------------------------------------------------
fold: 1
epoch: 2
loss: 0.894130551815033
training time 0h 2m 36s
-------

**Original Model with embedding size 768**

In [ ]:
!python3 run_seq2tree.py

Reading lines...
Transfer numbers...
(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['(', 'N1', '-', '1', ')', '*', 'N0'], ['2', '11'], [16, 34])
Indexing words...
keep_words 3671 / 9971 = 0.3682
Indexed 3674 words in input language, 23 words in output
Number of training data 18530
Number of testind data 4632
Vocabs Saved
TIL: 3674
([3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 10, 15, 1, 16, 17, 2, 10, 18, 19, 20, 21, 16, 22], 27, [0, 9, 8], 3, ['5', '0.8', '150'], [3, 12, 16], [])
fold: 1
epoch: 1
loss: 1.5688643767915924
training time 0h 2m 39s
--------------------------------
915 1021 4632
test_answer_acc 0.19753886010362695 0.22042314335060448
testing time 0h 5m 48s
------------------------------------------------------
fold: 1
epoch: 2
loss: 1.1292165809664232
training time 0h 2

**Generating Tokenized BERT embeddings for tokens in vocab**

In [ ]:
import json
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
import numpy as np
import pickle


vocab = {}
emb_dict = {}
with open("vocab_dict.json",'r') as f:
  vocab = json.load(f)

words_list = list(vocab.keys())

print(words_list[0])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

BertModel = BertModel.from_pretrained('bert-base-chinese').to(device)
BertModel.eval()

print(len(words_list))
for idx in range(2):
  print(idx)
  cap = u'[CLS] '+words_list[idx]
                  
  if cap != '\u3000' and cap != '':
    tokenized_cap = tokenizer.tokenize(cap)                
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_cap)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)

    with torch.no_grad():
      encoded_layers, _ = BertModel(tokens_tensor)

    bert_embedding = encoded_layers[11].squeeze(0)

    print(bert_embedding.shape)

    dim = bert_embedding.shape[0]

    if dim > 1:
      x = bert_embedding[0]
      x = x.reshape(1,768)
      for i in range(1,dim):
        temp_emb = bert_embedding[i]
        temp_emb = temp_emb.reshape(1,768)
        x = torch.add(x,temp_emb)
    else:
      x = bert_embedding[0]
      x = x.reshape(1,768)

    x = x.to("cpu")
    x_np = x.data.numpy()
  else:
    x_np = np.random.randn(1, 768)

  emb_dict[words_list[idx]] = x_np

with open('embeddings_dict.pickle', 'wb') as handle:
    pickle.dump(emb_dict, handle)



PAD
3690
0
torch.Size([2, 768])
1
torch.Size([3, 768])


In [ ]:
#Embedding size = 768, preloaded embeddings from BERT 
!python3 run_seq2tree.py

Reading lines...
Transfer numbers...
(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['(', 'N1', '-', '1', ')', '*', 'N0'], ['2', '11'], [16, 34])
Indexing words...
keep_words 3671 / 9971 = 0.3682
Indexed 3674 words in input language, 23 words in output
Number of training data 18530
Number of testind data 4632
Vocab Saved
TIL: 3674
([3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 10, 15, 1, 16, 17, 2, 10, 18, 19, 20, 21, 16, 22], 27, [0, 9, 8], 3, ['5', '0.8', '150'], [3, 12, 16], [])
Input size: 3674
Embeddings Loaded
fold: 1
epoch: 1
loss: 1.4425452291965484
training time 0h 2m 45s
--------------------------------
1050 1164 4632
test_answer_acc 0.2266839378238342 0.25129533678756477
testing time 0h 4m 40s
------------------------------------------------------
fold: 1
epoch: 2
loss: 0.8

In [ ]:
#Embedding size = 768, preloaded embeddings from BERT (Run 2) Epochs 20
!python3 run_seq2tree.py

Reading lines...
Transfer numbers...
(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['(', 'N1', '-', '1', ')', '*', 'N0'], ['2', '11'], [16, 34])
Indexing words...
keep_words 3671 / 9971 = 0.3682
Indexed 3674 words in input language, 23 words in output
Number of training data 18530
Number of testind data 4632
Vocab Saved
TIL: 3674
PAD
3674
([3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 10, 15, 1, 16, 17, 2, 10, 18, 19, 20, 21, 16, 22], 27, [0, 9, 8], 3, ['5', '0.8', '150'], [3, 12, 16], [])
Input size: 3674
Embeddings Loaded
fold: 1
epoch: 1
loss: 1.38668848193925
training time 0h 2m 29s
--------------------------------
1289 1424 4632
test_answer_acc 0.27828151986183075 0.307426597582038
testing time 0h 4m 28s
------------------------------------------------------
fold: 1
epoch: 2
los

In [ ]:
#Embedding size = 768, preloaded embeddings from BERT (Run 3) epochs 40
!python3 run_seq2tree.py

Reading lines...
Transfer numbers...
(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['(', 'N1', '-', '1', ')', '*', 'N0'], ['2', '11'], [16, 34])
Indexing words...
keep_words 3671 / 9971 = 0.3682
Indexed 3674 words in input language, 23 words in output
Number of training data 18530
Number of testind data 4632
Vocab Saved
TIL: 3674
PAD
100% 109540/109540 [00:00<00:00, 575587.92B/s]
100% 382072689/382072689 [00:09<00:00, 38913045.08B/s]
3674
([3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 10, 15, 1, 16, 17, 2, 10, 18, 19, 20, 21, 16, 22], 27, [0, 9, 8], 3, ['5', '0.8', '150'], [3, 12, 16], [])
Input size: 3674
Embeddings Loaded
fold: 1
epoch: 1
loss: 1.372649154169806
training time 0h 2m 45s
--------------------------------
1360 1500 4632
test_answer_acc 0.29360967184801384 0.32383419689

In [ ]:
#Embedding size = 768, preloaded both input and output embeddings from BERT epochs 80
!python3 run_seq2tree.py

Reading lines...
Transfer numbers...
(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['(', 'N1', '-', '1', ')', '*', 'N0'], ['2', '11'], [16, 34])
Indexing words...
keep_words 3671 / 9971 = 0.3682
Indexed 3674 words in input language, 23 words in output
Number of training data 18530
Number of testind data 4632
Vocabs Saved
TIL: 3674
PAD
3674
*
23
([3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 10, 15, 1, 16, 17, 2, 10, 18, 19, 20, 21, 16, 22], 27, [0, 9, 8], 3, ['5', '0.8', '150'], [3, 12, 16], [])
Input size: 3674
 Input Embeddings Loaded
Output Embeddings Loaded
fold: 1
epoch: 1
loss: 1.3815670584810191
training time 0h 2m 41s
--------------------------------
1232 1360 4632
test_answer_acc 0.2659758203799655 0.29360967184801384
testing time 0h 5m 8s
-----------------------------------

In [ ]:
#Embedding size = 768,  preloaded both input and output embeddings from BERT epochs 80
!python3 run_seq2tree.py

Reading lines...
Transfer numbers...
(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['(', 'N1', '-', '1', ')', '*', 'N0'], ['2', '11'], [16, 34])
Indexing words...
keep_words 3671 / 9971 = 0.3682
Indexed 3674 words in input language, 23 words in output
Number of training data 18530
Number of testind data 4632
Vocabs Saved
TIL: 3674
PAD
100% 109540/109540 [00:00<00:00, 16908109.53B/s]
100% 382072689/382072689 [00:08<00:00, 46184584.26B/s]
3674
*
23
([3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 10, 15, 1, 16, 17, 2, 10, 18, 19, 20, 21, 16, 22], 27, [0, 9, 8], 3, ['5', '0.8', '150'], [3, 12, 16], [])
Input size: 3674
 Input Embeddings Loaded
Output Embeddings Loaded
fold: 1
epoch: 1
loss: 1.5628295610690939
training time 0h 2m 42s
--------------------------------
682 746 4632
test_answe

**Trying out modification 2 to incorporate transformer encoder replacing the GRU**

In [ ]:
#Transformer encoder
!python3 run_seq2tree.py

Reading lines...
Transfer numbers...
(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['(', 'N1', '-', '1', ')', '*', 'N0'], ['2', '11'], [16, 34])
Indexing words...
keep_words 3671 / 9971 = 0.3682
Indexed 3674 words in input language, 23 words in output
Number of training data 18530
Number of testind data 4632
Vocabs Saved
TIL: 3674
PAD
3674
*
23
([3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 10, 15, 1, 16, 17, 2, 10, 18, 19, 20, 21, 16, 22], 27, [0, 9, 8], 3, ['5', '0.8', '150'], [3, 12, 16], [])
Input size: 3674
 Input Embeddings Loaded
Output Embeddings Loaded
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `op

In [ ]:
import torch
#multihead_attn = torch.nn.MultiheadAttention(embed_dim, num_heads)
print(torch.__version__)

1.0.1.post2


In [ ]:
from sklearn.manifold import TSNE

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.2MB/s 
     |████████████████████████████████| 2.9MB 17.4MB/s 
     |████████████████████████████████| 890kB 52.2MB/s 
     |████████████████████████████████| 1.1MB 53.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f2c5d66bb20e841c72c4922e54624ffe889fd1973872d071407c459d2c558e7b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import json
from transformers import BertTokenizer, BertModel
import torch

vocab = {}
with open("vocab_dict.json",'r') as f:
  vocab = json.load(f)

words_list = list(vocab.keys())

print(words_list[0])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

BertModel = BertModel.from_pretrained('bert-base-chinese', output_hidden_states = True).to(device)
BertModel.eval()

cap = words_list[0]
                
tokenized_cap = tokenizer.tokenize(cap)                
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_cap)
tokens_tensor = torch.tensor([indexed_tokens]).to(device)

with torch.no_grad():
  encoded_layers = BertModel(tokens_tensor)

bert_embedding = encoded_layers[2][11].squeeze(0)



#tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
#marked_text = "[CLS] " + words_list[0] + " [SEP]"
#tokenized_text = tokenizer.tokenize(marked_text)

#print(tokenized_text)

split_cap = cap.split()
tokens_embedding = []
j = 0

for full_token in split_cap:
    curr_token = ''
    x = 0
    for i,_ in enumerate(tokenized_cap[1:]): # disregard CLS
        token = tokenized_cap[i+j]
        print(token)
        piece_embedding = bert_embedding[i+j]
        
        # full token
        if token == full_token and curr_token == '' :
            tokens_embedding.append(piece_embedding)
            j += 1
            break
        else: # partial token
            x += 1
            
            if curr_token == '':
                tokens_embedding.append(piece_embedding)
                curr_token += token.replace('#', '')
            else:
                tokens_embedding[-1] = torch.add(tokens_embedding[-1], piece_embedding)
                curr_token += token.replace('#', '')
                
                if curr_token == full_token: # end of partial
                    j += x
                    break                            

cap_embedding = torch.stack(tokens_embedding)



时代
时


In [ ]:
import json
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
import numpy as np
import pickle


vocab = {}
emb_dict = {}
with open("vocab_dict.json",'r') as f:
  vocab = json.load(f)

words_list = list(vocab.keys())

print(words_list[0])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

BertModel = BertModel.from_pretrained('bert-base-chinese').to(device)
BertModel.eval()

print(len(words_list))
for idx in range(len(words_list)):
  print(idx)
  cap = words_list[idx]
                  
  if cap != '\u3000' and cap != '':
    tokenized_cap = tokenizer.tokenize(cap)                
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_cap)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)

    with torch.no_grad():
      encoded_layers, _ = BertModel(tokens_tensor)

    bert_embedding = encoded_layers[11].squeeze(0)

    dim = bert_embedding.shape[0]

    if dim > 1:
      x = bert_embedding[0]
      x = x.reshape(1,768)
      for i in range(1,dim):
        temp_emb = bert_embedding[i]
        temp_emb = temp_emb.reshape(1,768)
        x = torch.add(x,temp_emb)
    else:
      x = bert_embedding[0]
      x = x.reshape(1,768)

    x = x.to("cpu")
    x_np = x.data.numpy()
  else:
    x_np = np.random.randn(1, 768)

  emb_dict[words_list[idx]] = x_np

with open('embeddings_dict.pickle', 'wb') as handle:
    pickle.dump(emb_dict, handle)



PAD
3690
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
2

RuntimeError: ignored

In [ ]:
#bert_embedding = bert_embedding[1:]
print(bert_embedding[1:].shape)

torch.Size([2, 768])


In [ ]:
if len(indexed_tokens) == 0:
  print(indexed_tokens)


Blah []


In [ ]:
import pickle
import torch
import numpy as np

with open('embeddings_dict.pickle', 'rb') as handle:
    embb = pickle.load(handle)

embb_keys = list(embb.keys())

emp_emb = np.empty([len(embb_keys), 768])

for i in range(len(embb_keys)):
  key_val = embb[embb_keys[i]]
  emp_emb[i] = key_val

print(emp_emb.shape)

x_torch = torch.from_numpy(emp_emb)

(3674, 768)


In [ ]:
print(x_torch.shape)

torch.Size([3674, 768])


In [ ]:
emp_arr1 = np.empty([4, 4])
emp_arr1[0] = np.array([123000,100,100,100])

print(emp_arr1)

[[1.23000000e+005 1.00000000e+002 1.00000000e+002 1.00000000e+002]
 [1.03977794e-312 1.01855798e-312 1.23075756e-312 1.10343781e-312]
 [1.03977794e-312 9.76118064e-313 1.08221785e-312 1.18831764e-312]
 [1.06099790e-312 1.74490058e-015 4.64720292e-023 5.07136406e-018]]


In [ ]:
cap

''

In [ ]:
x = x.to("cpu")
x_np = x.data.numpy()

print(x_np.shape)

(1, 768)


In [ ]:
print(split_cap)
print(tokenized_cap)
print(bert_embedding[0].shape)

i1 = bert_embedding[0]
i1 = i1.reshape(1,768)

i1 = i1.to_numpy()
print(i1.shape)

['时代']
['时', '代']
torch.Size([768])


AttributeError: ignored